### Create simple neural sequence and save timestamps and activity pattern to file

In [1]:
from animation import build_frames, merge_frames, create_gif
from copy import copy
import numpy as np
import os
import shelve

DUR = 2
FS = 1000
V_REST = -0.07
V_TH = -0.04

STARTS = [100, 300, 500, 700]
ITVL = 200

ts = np.linspace(0, DUR, FS * DUR, endpoint=False)
vs = V_REST * np.ones((len(ts), len(STARTS)))
spikes = np.zeros(vs.shape, dtype=bool)

for neur, start in enumerate(STARTS):
    for offset in [0, 1000]:
        vs[offset+start:offset+start+ITVL, neur] = np.linspace(V_REST, V_TH, ITVL)
        spikes[offset+start+ITVL, neur] = True
        
xs = np.arange(len(STARTS))
ys = np.arange(len(STARTS))

positions = np.array([xs, ys])

# make sure save directory exists
if not os.path.exists('test_anim'): os.makedirs('test_anim')

data_t = shelve.open('test_anim/timestamp')
data_t['timestamps'] = ts
data_t['fs'] = FS
data_t.close()

data_a = shelve.open('test_anim/ntwk_activity')
data_a['vs'] = vs
data_a['spikes'] = spikes
data_a['v_rest'] = V_REST
data_a['v_th'] = V_TH
data_a['positions'] = positions
data_a.close()

### Build frames and gif

In [2]:
FPS = [30, 120, 60, 30]
PLAYBACK_FPS = [15, 30, 30, 30]

In [3]:
for fps, playback_fps in zip(FPS, PLAYBACK_FPS):
    save_file_gif = 'test_anim/ntwk_activity_{}_fps_{}Hz_playback'.format(
        fps, playback_fps)

    save_files = build_frames.ntwk_activity(
        save_prefix='test_anim/ntwk_activity_frames/frame',
        time_file='test_anim/timestamp',
        activity_file='test_anim/ntwk_activity',
        fps=fps,
        frames_per_spike=5,
        verbose=True)
    
    gif = create_gif(
        save_files, save_file=save_file_gif,
        playback_fps=playback_fps, verbose=True)

save_files_ntwk = copy(save_files)
gif_ntwk = gif

Using timestamp file "test_anim/timestamp" and activity file "test_anim/ntwk_activity".
Frames will be saved with prefix "test_anim/ntwk_activity_frames/frame".
Loading timestamps and network activity data...
Data loaded.
Downsampling data from 1000 Hz to 30 fps...
Data downsampled.
Generating and saving 60 frames...
Frames saved.
Loading source images...
Source images loaded.
Generating gif...
Gif generated.
Using timestamp file "test_anim/timestamp" and activity file "test_anim/ntwk_activity".
Frames will be saved with prefix "test_anim/ntwk_activity_frames/frame".
Loading timestamps and network activity data...
Data loaded.
Downsampling data from 1000 Hz to 120 fps...
Data downsampled.
Generating and saving 240 frames...
Frames saved.
Loading source images...
Source images loaded.
Generating gif...
Gif generated.
Using timestamp file "test_anim/timestamp" and activity file "test_anim/ntwk_activity".
Frames will be saved with prefix "test_anim/ntwk_activity_frames/frame".
Loading tim

### Display resulting example GIF

In [4]:
from IPython import display

print(gif_ntwk)
display.HTML('<img src="{}" />'.format(gif_ntwk))

test_anim/ntwk_activity_30_fps_30Hz_playback.gif


# Create position sequence for trajectory animation

In [5]:
from scipy.ndimage import gaussian_filter1d

STD = 1
SMOOTHING = 200
SEED = 0

np.random.seed(SEED)

# make random x and y position sequences
xys = STD * np.random.randn(len(ts), 2)
# smooth them
xys = gaussian_filter1d(xys, SMOOTHING, axis=0)

# make sure save directory exists
if not os.path.exists('test_anim'): os.makedirs('test_anim')

# save traj file
data_tr = shelve.open('test_anim/traj')
data_tr['xys'] = xys
data_tr.close()

### Build frames and gif

In [6]:
for fps, playback_fps in zip(FPS, PLAYBACK_FPS):
    save_file_gif = 'test_anim/traj_{}_fps_{}Hz_playback'.format(
        fps, playback_fps)

    save_files = build_frames.traj(
        save_prefix='test_anim/traj_frames/frame',
        time_file='test_anim/timestamp',
        traj_file='test_anim/traj',
        fps=fps,
        verbose=True)

    gif = create_gif(
        save_files, save_file=save_file_gif,
        playback_fps=playback_fps, verbose=True)

save_files_traj = copy(save_files)
gif_traj = gif

Loading source images...
Source images loaded.
Generating gif...
Gif generated.
Loading source images...
Source images loaded.
Generating gif...
Gif generated.
Loading source images...
Source images loaded.
Generating gif...
Gif generated.
Loading source images...
Source images loaded.
Generating gif...
Gif generated.


In [7]:
print(gif_traj)
display.HTML('<img src="{}" />'.format(gif_traj))

test_anim/traj_30_fps_30Hz_playback.gif


# Create merged frames and gif

In [8]:
save_prefix = 'test_anim/merged_frames/frame'
rects = [(0, 0, 432, 288), (432, 0, 864, 288)]
size = (864, 288)

merged_frames = merge_frames([save_files_ntwk, save_files_traj], save_prefix, rects, size)

gif_merged = create_gif(merged_frames, 'test_anim/merged', playback_fps=30, verbose=True)

Loading source images...
Source images loaded.
Generating gif...
Gif generated.


In [11]:
print(gif_merged)
display.HTML('<img src="{}" />'.format(gif_merged))

test_anim/merged.gif
